# DC Council Tax Reforms Analysis (2025-2026)

This notebook analyzes the tax reforms approved by the DC City Council:

## Reform 1: EITC Enhancement (2025)
- Increase DC EITC match from 85% to 100% of federal EITC
- Effective: Tax year 2025

## Reform 2: Child Tax Credit (2026)
- $1,000 per qualifying child under 18
- Income thresholds:
  - Single/HOH/Widow: $55,000
  - Joint filers: $70,000
  - Married filing separately: $35,000
- Phase-out: $50 reduction per $1,000 over threshold
- Fully refundable
- Effective: Tax year 2026

## Two Baseline Scenarios

### Baseline A: Current Law
Keeps existing DC CTC ($420/child under 6, higher income thresholds)

### Baseline B: Zeroed Out
Eliminates existing DC CTC (reflects budget removal before this amendment)

## Analysis Structure
For each baseline:
1. 2025: EITC reform only
2. 2026: CTC reform only
3. 2026: Combined EITC + CTC impact

In [21]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

## Helper Functions for Impact Analysis

In [ ]:
def calculate_child_poverty(sim, period=2025):
    """
    Calculate overall child poverty rate (children under 18).
    Returns poverty rate, count in poverty, and total children.
    """
    # Get person-level data
    age = sim.calculate("age", period=period)
    is_in_poverty = sim.calculate("person_in_poverty", period=period)
    
    # Filter for children under 18
    mask = age < 18
    
    # Calculate poverty count and total children (unweighted)
    children_in_poverty = is_in_poverty[mask].sum()
    total_children = mask.sum()
    
    # Calculate poverty rate
    poverty_rate = children_in_poverty / total_children if total_children > 0 else 0
    
    return {
        "poverty_rate": poverty_rate,
        "children_in_poverty": children_in_poverty,
        "total_children": total_children
    }

def calculate_budgetary_impact(baseline_sim, reform_sim, variable, period=2025):
    """
    Calculate the budgetary impact of a reform.
    """
    baseline_value = baseline_sim.calculate(variable, period=period, map_to="household").sum()
    reform_value = reform_sim.calculate(variable, period=period, map_to="household").sum()
    
    return {
        "baseline": baseline_value,
        "reform": reform_value,
        "net_cost": reform_value - baseline_value
    }

def format_currency(value):
    """Format value as currency in millions."""
    return f"${value/1e6:.2f}M"

def format_percent(value):
    """Format value as percentage."""
    return f"{value*100:.2f}%"

## Setup Baselines

In [23]:
# Baseline A: Current Law (default - keeps existing DC CTC)
print("Loading Baseline A: Current Law (keeps existing DC CTC)...")
baseline_current_law = Microsimulation(dataset='hf://policyengine/test/DC.h5')

# Check current CTC cost
current_ctc = baseline_current_law.calculate("dc_ctc", period=2025, map_to="household")
print(f"  Current DC CTC cost: {format_currency(current_ctc.sum())}")
print("  Baseline A loaded!")

Loading Baseline A: Current Law (keeps existing DC CTC)...
  Current DC CTC cost: $16.70M
  Baseline A loaded!


In [24]:
# Baseline B: Zeroed Out (eliminates existing DC CTC)
def create_zero_ctc_baseline():
    """Zero out the existing DC CTC (reflects budget removal)"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": 0
            }
        },
        country_id="us",
    )

print("Loading Baseline B: Zeroed Out (eliminates existing DC CTC)...")
zero_ctc_reform = create_zero_ctc_baseline()
baseline_zeroed_out = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=zero_ctc_reform)

# Verify CTC is zeroed out
zeroed_ctc = baseline_zeroed_out.calculate("dc_ctc", period=2025, map_to="household")
print(f"  Zeroed DC CTC cost: {format_currency(zeroed_ctc.sum())}")
print("  Baseline B loaded!")

Loading Baseline B: Zeroed Out (eliminates existing DC CTC)...
  Zeroed DC CTC cost: $0.00M
  Baseline B loaded!


## Define Reform Functions

In [25]:
def create_eitc_reform():
    """Increase DC EITC match from 85% to 100%"""
    return Reform.from_dict(
        {
            "gov.states.dc.tax.income.credits.eitc.with_children.match": {
                "2025-01-01.2100-12-31": 1.0  # 100% match
            }
        },
        country_id="us",
    )

def create_new_ctc_reform():
    """Create the new DC CTC as specified in the amendment"""
    return Reform.from_dict(
        {
            # Set amount to $1,000 per child
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2026-01-01.2100-12-31": 1000
            },
            # Age threshold: under 18
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2026-01-01.2100-12-31": 18
            },
            # No child cap (unlimited)
            "gov.states.dc.tax.income.credits.ctc.child.child_cap": {
                "2026-01-01.2100-12-31": 999
            },
            # Phase-out: $50 per $1,000 over threshold
            "gov.states.dc.tax.income.credits.ctc.phase_out.amount": {
                "2026-01-01.2100-12-31": 50
            },
            # Income thresholds by filing status
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2026-01-01.2100-12-31": 70_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2026-01-01.2100-12-31": 35_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2026-01-01.2100-12-31": 55_000
            }
        },
        country_id="us",
    )

def create_combined_reform():
    """Create combined EITC + CTC reform"""
    return Reform.from_dict(
        {
            # EITC: 100% match
            "gov.states.dc.tax.income.credits.eitc.with_children.match": {
                "2025-01-01.2100-12-31": 1.0
            },
            # CTC: $1,000 per child
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2026-01-01.2100-12-31": 1000
            },
            "gov.states.dc.tax.income.credits.ctc.child.age_threshold": {
                "2026-01-01.2100-12-31": 18
            },
            "gov.states.dc.tax.income.credits.ctc.child.child_cap": {
                "2026-01-01.2100-12-31": 999
            },
            "gov.states.dc.tax.income.credits.ctc.phase_out.amount": {
                "2026-01-01.2100-12-31": 50
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SINGLE": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.JOINT": {
                "2026-01-01.2100-12-31": 70_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SEPARATE": {
                "2026-01-01.2100-12-31": 35_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.HEAD_OF_HOUSEHOLD": {
                "2026-01-01.2100-12-31": 55_000
            },
            "gov.states.dc.tax.income.credits.ctc.income_threshold.SURVIVING_SPOUSE": {
                "2026-01-01.2100-12-31": 55_000
            }
        },
        country_id="us",
    )

def create_eitc_and_zero_ctc():
    """EITC enhancement + zero out CTC (for zeroed baseline scenario)"""
    return Reform.from_dict(
        {
            # EITC: 100% match
            "gov.states.dc.tax.income.credits.eitc.with_children.match": {
                "2025-01-01.2100-12-31": 1.0
            },
            # Zero out CTC
            "gov.states.dc.tax.income.credits.ctc.amount": {
                "2025-01-01.2100-12-31": 0
            }
        },
        country_id="us",
    )

print("Reform functions defined!")

Reform functions defined!


# BASELINE A: CURRENT LAW

Analysis assuming the existing DC CTC ($420/child under 6) remains in place

## A1. EITC Reform (2025) vs Current Law

In [26]:
print("Creating EITC reform simulation (current law baseline)...")
eitc_reform = create_eitc_reform()
eitc_reform_currentlaw = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=eitc_reform)
print("Done!")

Creating EITC reform simulation (current law baseline)...
Done!


In [27]:
print("=" * 80)
print("BASELINE A - EITC REFORM IMPACTS (2025)")
print("=" * 80)

# Budgetary impact
a1_eitc_budget = calculate_budgetary_impact(baseline_current_law, eitc_reform_currentlaw, "dc_eitc", period=2025)
print(f"\nEITC Budgetary Impact:")
print(f"  Baseline DC EITC cost: {format_currency(a1_eitc_budget['baseline'])}")
print(f"  Reform DC EITC cost:   {format_currency(a1_eitc_budget['reform'])}")
print(f"  Net cost:              {format_currency(a1_eitc_budget['net_cost'])}")

# CTC remains unchanged in 2025
a1_ctc_budget = calculate_budgetary_impact(baseline_current_law, eitc_reform_currentlaw, "dc_ctc", period=2025)
print(f"\nCTC Budgetary Impact (unchanged):")
print(f"  DC CTC cost: {format_currency(a1_ctc_budget['baseline'])}")

# Poverty impact
a1_baseline_poverty = calculate_child_poverty(baseline_current_law, period=2025)
a1_eitc_poverty = calculate_child_poverty(eitc_reform_currentlaw, period=2025)

a1_poverty_reduction = a1_baseline_poverty['poverty_rate'] - a1_eitc_poverty['poverty_rate']
a1_poverty_pct_reduction = (a1_poverty_reduction / a1_baseline_poverty['poverty_rate']) * 100

print(f"\nPoverty Impact (Children Under 18):")
print(f"  Baseline poverty rate: {format_percent(a1_baseline_poverty['poverty_rate'])}")
print(f"  Reform poverty rate:   {format_percent(a1_eitc_poverty['poverty_rate'])}")
print(f"  Absolute reduction:    {format_percent(a1_poverty_reduction)}")
print(f"  Relative reduction:    {a1_poverty_pct_reduction:.2f}%")
print(f"  Children lifted out:   {int(a1_baseline_poverty['children_in_poverty'] - a1_eitc_poverty['children_in_poverty']):,}")

BASELINE A - EITC REFORM IMPACTS (2025)

EITC Budgetary Impact:
  Baseline DC EITC cost: $57.63M
  Reform DC EITC cost:   $64.15M
  Net cost:              $6.52M

CTC Budgetary Impact (unchanged):
  DC CTC cost: $16.70M

Poverty Impact (Children Under 18):
  Baseline poverty rate: 28.71%
  Reform poverty rate:   28.71%
  Absolute reduction:    0.00%
  Relative reduction:    0.00%
  Children lifted out:   0


## A2. New CTC Only (2026) vs Current Law

In [28]:
print("Creating new CTC reform simulation (current law baseline)...")
new_ctc_reform = create_new_ctc_reform()
new_ctc_reform_currentlaw = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=new_ctc_reform)
print("Done!")

Creating new CTC reform simulation (current law baseline)...
Done!


In [29]:
print("=" * 80)
print("BASELINE A - NEW CTC REFORM IMPACTS (2026) - CTC Only")
print("=" * 80)

# Budgetary impact - EITC (baseline 85%)
a2_eitc_budget = calculate_budgetary_impact(baseline_current_law, new_ctc_reform_currentlaw, "dc_eitc", period=2026)
print(f"\nEITC Budgetary Impact (unchanged at 85%):")
print(f"  DC EITC cost: {format_currency(a2_eitc_budget['baseline'])}")

# Budgetary impact - CTC
a2_ctc_budget = calculate_budgetary_impact(baseline_current_law, new_ctc_reform_currentlaw, "dc_ctc", period=2026)
print(f"\nCTC Budgetary Impact:")
print(f"  Baseline DC CTC cost: {format_currency(a2_ctc_budget['baseline'])} (old CTC: $420, under 6)")
print(f"  Reform DC CTC cost:   {format_currency(a2_ctc_budget['reform'])} (new CTC: $1000, under 18)")
print(f"  Net cost:             {format_currency(a2_ctc_budget['net_cost'])}")

# Poverty impact
a2_baseline_poverty = calculate_child_poverty(baseline_current_law, period=2026)
a2_ctc_poverty = calculate_child_poverty(new_ctc_reform_currentlaw, period=2026)

a2_poverty_reduction = a2_baseline_poverty['poverty_rate'] - a2_ctc_poverty['poverty_rate']
a2_poverty_pct_reduction = (a2_poverty_reduction / a2_baseline_poverty['poverty_rate']) * 100

print(f"\nPoverty Impact (Children Under 18):")
print(f"  Baseline poverty rate: {format_percent(a2_baseline_poverty['poverty_rate'])}")
print(f"  Reform poverty rate:   {format_percent(a2_ctc_poverty['poverty_rate'])}")
print(f"  Absolute reduction:    {format_percent(a2_poverty_reduction)}")
print(f"  Relative reduction:    {a2_poverty_pct_reduction:.2f}%")
print(f"  Children lifted out:   {int(a2_baseline_poverty['children_in_poverty'] - a2_ctc_poverty['children_in_poverty']):,}")

BASELINE A - NEW CTC REFORM IMPACTS (2026) - CTC Only

EITC Budgetary Impact (unchanged at 85%):
  DC EITC cost: $57.78M

CTC Budgetary Impact:
  Baseline DC CTC cost: $16.82M (old CTC: $420, under 6)
  Reform DC CTC cost:   $85.78M (new CTC: $1000, under 18)
  Net cost:             $68.96M

Poverty Impact (Children Under 18):
  Baseline poverty rate: 28.29%
  Reform poverty rate:   27.69%
  Absolute reduction:    0.60%
  Relative reduction:    2.11%
  Children lifted out:   223,716


## A3. Combined EITC + New CTC (2026) vs Current Law

In [30]:
print("Creating combined EITC + CTC reform simulation (current law baseline)...")
combined_reform = create_combined_reform()
combined_reform_currentlaw = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=combined_reform)
print("Done!")

Creating combined EITC + CTC reform simulation (current law baseline)...
Done!


In [31]:
print("=" * 80)
print("BASELINE A - COMBINED REFORM IMPACTS (2026) - EITC + CTC")
print("=" * 80)

# Budgetary impact - EITC
a3_eitc_budget = calculate_budgetary_impact(baseline_current_law, combined_reform_currentlaw, "dc_eitc", period=2026)
print(f"\nEITC Budgetary Impact:")
print(f"  Baseline DC EITC cost: {format_currency(a3_eitc_budget['baseline'])} (85% match)")
print(f"  Reform DC EITC cost:   {format_currency(a3_eitc_budget['reform'])} (100% match)")
print(f"  Net cost:              {format_currency(a3_eitc_budget['net_cost'])}")

# Budgetary impact - CTC
a3_ctc_budget = calculate_budgetary_impact(baseline_current_law, combined_reform_currentlaw, "dc_ctc", period=2026)
print(f"\nCTC Budgetary Impact:")
print(f"  Baseline DC CTC cost: {format_currency(a3_ctc_budget['baseline'])} (old CTC: $420, under 6)")
print(f"  Reform DC CTC cost:   {format_currency(a3_ctc_budget['reform'])} (new CTC: $1000, under 18)")
print(f"  Net cost:             {format_currency(a3_ctc_budget['net_cost'])}")

# Total budgetary impact
a3_total_cost = a3_eitc_budget['net_cost'] + a3_ctc_budget['net_cost']
print(f"\nTotal Combined Net Cost: {format_currency(a3_total_cost)}")

# Poverty impact
a3_baseline_poverty = calculate_child_poverty(baseline_current_law, period=2026)
a3_combined_poverty = calculate_child_poverty(combined_reform_currentlaw, period=2026)

a3_poverty_reduction = a3_baseline_poverty['poverty_rate'] - a3_combined_poverty['poverty_rate']
a3_poverty_pct_reduction = (a3_poverty_reduction / a3_baseline_poverty['poverty_rate']) * 100

print(f"\nPoverty Impact (Children Under 18):")
print(f"  Baseline poverty rate: {format_percent(a3_baseline_poverty['poverty_rate'])}")
print(f"  Reform poverty rate:   {format_percent(a3_combined_poverty['poverty_rate'])}")
print(f"  Absolute reduction:    {format_percent(a3_poverty_reduction)}")
print(f"  Relative reduction:    {a3_poverty_pct_reduction:.2f}%")
print(f"  Children lifted out:   {int(a3_baseline_poverty['children_in_poverty'] - a3_combined_poverty['children_in_poverty']):,}")

BASELINE A - COMBINED REFORM IMPACTS (2026) - EITC + CTC

EITC Budgetary Impact:
  Baseline DC EITC cost: $57.78M (85% match)
  Reform DC EITC cost:   $64.33M (100% match)
  Net cost:              $6.55M

CTC Budgetary Impact:
  Baseline DC CTC cost: $16.82M (old CTC: $420, under 6)
  Reform DC CTC cost:   $85.78M (new CTC: $1000, under 18)
  Net cost:             $68.96M

Total Combined Net Cost: $75.51M

Poverty Impact (Children Under 18):
  Baseline poverty rate: 28.29%
  Reform poverty rate:   27.69%
  Absolute reduction:    0.60%
  Relative reduction:    2.11%
  Children lifted out:   223,716


# BASELINE B: ZEROED OUT

Analysis assuming the existing DC CTC was eliminated (as in the recent budget)

## B1. EITC Reform (2025) vs Zeroed Out Baseline

In [32]:
print("Creating EITC reform simulation (zeroed out baseline)...")
eitc_and_zero_reform = create_eitc_and_zero_ctc()
eitc_reform_zeroed = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=eitc_and_zero_reform)
print("Done!")

Creating EITC reform simulation (zeroed out baseline)...
Done!


In [33]:
print("=" * 80)
print("BASELINE B - EITC REFORM IMPACTS (2025)")
print("=" * 80)

# Budgetary impact
b1_eitc_budget = calculate_budgetary_impact(baseline_zeroed_out, eitc_reform_zeroed, "dc_eitc", period=2025)
print(f"\nEITC Budgetary Impact:")
print(f"  Baseline DC EITC cost: {format_currency(b1_eitc_budget['baseline'])}")
print(f"  Reform DC EITC cost:   {format_currency(b1_eitc_budget['reform'])}")
print(f"  Net cost:              {format_currency(b1_eitc_budget['net_cost'])}")

# CTC is zero in both scenarios
b1_ctc_budget = calculate_budgetary_impact(baseline_zeroed_out, eitc_reform_zeroed, "dc_ctc", period=2025)
print(f"\nCTC Budgetary Impact (eliminated):")
print(f"  DC CTC cost: {format_currency(b1_ctc_budget['baseline'])}")

# Poverty impact
b1_baseline_poverty = calculate_child_poverty(baseline_zeroed_out, period=2025)
b1_eitc_poverty = calculate_child_poverty(eitc_reform_zeroed, period=2025)

b1_poverty_reduction = b1_baseline_poverty['poverty_rate'] - b1_eitc_poverty['poverty_rate']
b1_poverty_pct_reduction = (b1_poverty_reduction / b1_baseline_poverty['poverty_rate']) * 100

print(f"\nPoverty Impact (Children Under 18):")
print(f"  Baseline poverty rate: {format_percent(b1_baseline_poverty['poverty_rate'])}")
print(f"  Reform poverty rate:   {format_percent(b1_eitc_poverty['poverty_rate'])}")
print(f"  Absolute reduction:    {format_percent(b1_poverty_reduction)}")
print(f"  Relative reduction:    {b1_poverty_pct_reduction:.2f}%")
print(f"  Children lifted out:   {int(b1_baseline_poverty['children_in_poverty'] - b1_eitc_poverty['children_in_poverty']):,}")

BASELINE B - EITC REFORM IMPACTS (2025)

EITC Budgetary Impact:
  Baseline DC EITC cost: $57.63M
  Reform DC EITC cost:   $64.15M
  Net cost:              $6.52M

CTC Budgetary Impact (eliminated):
  DC CTC cost: $0.00M

Poverty Impact (Children Under 18):
  Baseline poverty rate: 28.71%
  Reform poverty rate:   28.71%
  Absolute reduction:    0.00%
  Relative reduction:    0.00%
  Children lifted out:   0


## B2. New CTC Only (2026) vs Zeroed Out Baseline

In [34]:
print("Creating new CTC reform simulation (zeroed out baseline)...")
# Apply the new CTC reform directly (it will overwrite the zeroed CTC)
new_ctc_reform_zeroed = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=new_ctc_reform)
print("Done!")

Creating new CTC reform simulation (zeroed out baseline)...
Done!


In [35]:
print("=" * 80)
print("BASELINE B - NEW CTC REFORM IMPACTS (2026) - CTC Only")
print("=" * 80)

# Budgetary impact - EITC (baseline 85%, unchanged)
b2_eitc_budget = calculate_budgetary_impact(baseline_zeroed_out, new_ctc_reform_zeroed, "dc_eitc", period=2026)
print(f"\nEITC Budgetary Impact (unchanged at 85%):")
print(f"  DC EITC cost: {format_currency(b2_eitc_budget['baseline'])}")

# Budgetary impact - CTC
b2_ctc_budget = calculate_budgetary_impact(baseline_zeroed_out, new_ctc_reform_zeroed, "dc_ctc", period=2026)
print(f"\nCTC Budgetary Impact:")
print(f"  Baseline DC CTC cost: {format_currency(b2_ctc_budget['baseline'])} (eliminated)")
print(f"  Reform DC CTC cost:   {format_currency(b2_ctc_budget['reform'])} (new CTC: $1000, under 18)")
print(f"  Net cost:             {format_currency(b2_ctc_budget['net_cost'])}")

# Poverty impact
b2_baseline_poverty = calculate_child_poverty(baseline_zeroed_out, period=2026)
b2_ctc_poverty = calculate_child_poverty(new_ctc_reform_zeroed, period=2026)

b2_poverty_reduction = b2_baseline_poverty['poverty_rate'] - b2_ctc_poverty['poverty_rate']
b2_poverty_pct_reduction = (b2_poverty_reduction / b2_baseline_poverty['poverty_rate']) * 100

print(f"\nPoverty Impact (Children Under 18):")
print(f"  Baseline poverty rate: {format_percent(b2_baseline_poverty['poverty_rate'])}")
print(f"  Reform poverty rate:   {format_percent(b2_ctc_poverty['poverty_rate'])}")
print(f"  Absolute reduction:    {format_percent(b2_poverty_reduction)}")
print(f"  Relative reduction:    {b2_poverty_pct_reduction:.2f}%")
print(f"  Children lifted out:   {int(b2_baseline_poverty['children_in_poverty'] - b2_ctc_poverty['children_in_poverty']):,}")

BASELINE B - NEW CTC REFORM IMPACTS (2026) - CTC Only

EITC Budgetary Impact (unchanged at 85%):
  DC EITC cost: $57.78M

CTC Budgetary Impact:
  Baseline DC CTC cost: $0.00M (eliminated)
  Reform DC CTC cost:   $85.78M (new CTC: $1000, under 18)
  Net cost:             $85.78M

Poverty Impact (Children Under 18):
  Baseline poverty rate: 28.29%
  Reform poverty rate:   27.69%
  Absolute reduction:    0.60%
  Relative reduction:    2.11%
  Children lifted out:   223,716


## B3. Combined EITC + New CTC (2026) vs Zeroed Out Baseline

In [36]:
print("Creating combined EITC + CTC reform simulation (zeroed out baseline)...")
# Apply the combined reform directly (it will set all parameters)
combined_reform_zeroed = Microsimulation(dataset='hf://policyengine/test/DC.h5', reform=combined_reform)
print("Done!")

Creating combined EITC + CTC reform simulation (zeroed out baseline)...
Done!


In [37]:
print("=" * 80)
print("BASELINE B - COMBINED REFORM IMPACTS (2026) - EITC + CTC")
print("=" * 80)

# Budgetary impact - EITC
b3_eitc_budget = calculate_budgetary_impact(baseline_zeroed_out, combined_reform_zeroed, "dc_eitc", period=2026)
print(f"\nEITC Budgetary Impact:")
print(f"  Baseline DC EITC cost: {format_currency(b3_eitc_budget['baseline'])} (85% match)")
print(f"  Reform DC EITC cost:   {format_currency(b3_eitc_budget['reform'])} (100% match)")
print(f"  Net cost:              {format_currency(b3_eitc_budget['net_cost'])}")

# Budgetary impact - CTC
b3_ctc_budget = calculate_budgetary_impact(baseline_zeroed_out, combined_reform_zeroed, "dc_ctc", period=2026)
print(f"\nCTC Budgetary Impact:")
print(f"  Baseline DC CTC cost: {format_currency(b3_ctc_budget['baseline'])} (eliminated)")
print(f"  Reform DC CTC cost:   {format_currency(b3_ctc_budget['reform'])} (new CTC: $1000, under 18)")
print(f"  Net cost:             {format_currency(b3_ctc_budget['net_cost'])}")

# Total budgetary impact
b3_total_cost = b3_eitc_budget['net_cost'] + b3_ctc_budget['net_cost']
print(f"\nTotal Combined Net Cost: {format_currency(b3_total_cost)}")

# Poverty impact
b3_baseline_poverty = calculate_child_poverty(baseline_zeroed_out, period=2026)
b3_combined_poverty = calculate_child_poverty(combined_reform_zeroed, period=2026)

b3_poverty_reduction = b3_baseline_poverty['poverty_rate'] - b3_combined_poverty['poverty_rate']
b3_poverty_pct_reduction = (b3_poverty_reduction / b3_baseline_poverty['poverty_rate']) * 100

print(f"\nPoverty Impact (Children Under 18):")
print(f"  Baseline poverty rate: {format_percent(b3_baseline_poverty['poverty_rate'])}")
print(f"  Reform poverty rate:   {format_percent(b3_combined_poverty['poverty_rate'])}")
print(f"  Absolute reduction:    {format_percent(b3_poverty_reduction)}")
print(f"  Relative reduction:    {b3_poverty_pct_reduction:.2f}%")
print(f"  Children lifted out:   {int(b3_baseline_poverty['children_in_poverty'] - b3_combined_poverty['children_in_poverty']):,}")

BASELINE B - COMBINED REFORM IMPACTS (2026) - EITC + CTC

EITC Budgetary Impact:
  Baseline DC EITC cost: $57.78M (85% match)
  Reform DC EITC cost:   $64.33M (100% match)
  Net cost:              $6.55M

CTC Budgetary Impact:
  Baseline DC CTC cost: $0.00M (eliminated)
  Reform DC CTC cost:   $85.78M (new CTC: $1000, under 18)
  Net cost:             $85.78M

Total Combined Net Cost: $92.33M

Poverty Impact (Children Under 18):
  Baseline poverty rate: 28.29%
  Reform poverty rate:   27.69%
  Absolute reduction:    0.60%
  Relative reduction:    2.11%
  Children lifted out:   223,716


# COMPREHENSIVE SUMMARY TABLES

## Summary: Baseline A (Current Law)

In [43]:
summary_a_data = [
    {
        "Scenario": "Baseline",
        "Year": "2025",
        "EITC Cost": format_currency(a1_eitc_budget['baseline']),
        "CTC Cost": format_currency(a1_ctc_budget['baseline']),
        "Total Cost": format_currency(a1_eitc_budget['baseline'] + a1_ctc_budget['baseline']),
        "Child Poverty Rate": format_percent(a1_baseline_poverty['poverty_rate']),
        "Children in Poverty": f"{int(a1_baseline_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "EITC Reform",
        "Year": "2025",
        "EITC Cost": format_currency(a1_eitc_budget['reform']),
        "CTC Cost": format_currency(a1_ctc_budget['baseline']),
        "Total Cost": format_currency(a1_eitc_budget['reform'] + a1_ctc_budget['baseline']),
        "Child Poverty Rate": format_percent(a1_eitc_poverty['poverty_rate']),
        "Children in Poverty": f"{int(a1_eitc_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "Baseline",
        "Year": "2026",
        "EITC Cost": format_currency(a2_eitc_budget['baseline']),
        "CTC Cost": format_currency(a2_ctc_budget['baseline']),
        "Total Cost": format_currency(a2_eitc_budget['baseline'] + a2_ctc_budget['baseline']),
        "Child Poverty Rate": format_percent(a2_baseline_poverty['poverty_rate']),
        "Children in Poverty": f"{int(a2_baseline_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "New CTC Only",
        "Year": "2026",
        "EITC Cost": format_currency(a2_eitc_budget['baseline']),
        "CTC Cost": format_currency(a2_ctc_budget['reform']),
        "Total Cost": format_currency(a2_eitc_budget['baseline'] + a2_ctc_budget['reform']),
        "Child Poverty Rate": format_percent(a2_ctc_poverty['poverty_rate']),
        "Children in Poverty": f"{int(a2_ctc_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "EITC + CTC",
        "Year": "2026",
        "EITC Cost": format_currency(a3_eitc_budget['reform']),
        "CTC Cost": format_currency(a3_ctc_budget['reform']),
        "Total Cost": format_currency(a3_eitc_budget['reform'] + a3_ctc_budget['reform']),
        "Child Poverty Rate": format_percent(a3_combined_poverty['poverty_rate']),
        "Children in Poverty": f"{int(a3_combined_poverty['children_in_poverty']):,}"
    }
]

summary_a_df = pd.DataFrame(summary_a_data)

print("\n" + "=" * 120)
print("BASELINE A SUMMARY: Current Law (existing CTC $420/child under 6 remains)")
print("=" * 120)
print(summary_a_df.to_string(index=False))

summary_a_df.to_csv("dc_reforms_baseline_a_current_law.csv", index=False)
print("\nExported to: dc_reforms_baseline_a_current_law.csv")


BASELINE A SUMMARY: Current Law (existing CTC $420/child under 6 remains)
    Scenario Year EITC Cost CTC Cost Total Cost Child Poverty Rate Children in Poverty
    Baseline 2025   $57.63M  $16.70M    $74.33M             28.71%          10,626,794
 EITC Reform 2025   $64.15M  $16.70M    $80.85M             28.71%          10,626,794
    Baseline 2026   $57.78M  $16.82M    $74.60M             28.29%          10,620,556
New CTC Only 2026   $57.78M  $85.78M   $143.56M             27.69%          10,396,840
  EITC + CTC 2026   $64.33M  $85.78M   $150.10M             27.69%          10,396,840

Exported to: dc_reforms_baseline_a_current_law.csv


## Summary: Baseline B (Zeroed Out)

In [42]:
summary_b_data = [
    {
        "Scenario": "Baseline",
        "Year": "2025",
        "EITC Cost": format_currency(b1_eitc_budget['baseline']),
        "CTC Cost": format_currency(b1_ctc_budget['baseline']),
        "Total Cost": format_currency(b1_eitc_budget['baseline'] + b1_ctc_budget['baseline']),
        "Child Poverty Rate": format_percent(b1_baseline_poverty['poverty_rate']),
        "Children in Poverty": f"{int(b1_baseline_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "EITC Reform",
        "Year": "2025",
        "EITC Cost": format_currency(b1_eitc_budget['reform']),
        "CTC Cost": format_currency(b1_ctc_budget['baseline']),
        "Total Cost": format_currency(b1_eitc_budget['reform'] + b1_ctc_budget['baseline']),
        "Child Poverty Rate": format_percent(b1_eitc_poverty['poverty_rate']),
        "Children in Poverty": f"{int(b1_eitc_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "Baseline",
        "Year": "2026",
        "EITC Cost": format_currency(b2_eitc_budget['baseline']),
        "CTC Cost": format_currency(b2_ctc_budget['baseline']),
        "Total Cost": format_currency(b2_eitc_budget['baseline'] + b2_ctc_budget['baseline']),
        "Child Poverty Rate": format_percent(b2_baseline_poverty['poverty_rate']),
        "Children in Poverty": f"{int(b2_baseline_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "New CTC Only",
        "Year": "2026",
        "EITC Cost": format_currency(b2_eitc_budget['baseline']),
        "CTC Cost": format_currency(b2_ctc_budget['reform']),
        "Total Cost": format_currency(b2_eitc_budget['baseline'] + b2_ctc_budget['reform']),
        "Child Poverty Rate": format_percent(b2_ctc_poverty['poverty_rate']),
        "Children in Poverty": f"{int(b2_ctc_poverty['children_in_poverty']):,}"
    },
    {
        "Scenario": "EITC + CTC",
        "Year": "2026",
        "EITC Cost": format_currency(b3_eitc_budget['reform']),
        "CTC Cost": format_currency(b3_ctc_budget['reform']),
        "Total Cost": format_currency(b3_eitc_budget['reform'] + b3_ctc_budget['reform']),
        "Child Poverty Rate": format_percent(b3_combined_poverty['poverty_rate']),
        "Children in Poverty": f"{int(b3_combined_poverty['children_in_poverty']):,}"
    }
]

summary_b_df = pd.DataFrame(summary_b_data)

print("\n" + "=" * 120)
print("BASELINE B SUMMARY: Zeroed Out (existing CTC eliminated in budget)")
print("=" * 120)
print(summary_b_df.to_string(index=False))

summary_b_df.to_csv("dc_reforms_baseline_b_zeroed_out.csv", index=False)
print("\nExported to: dc_reforms_baseline_b_zeroed_out.csv")


BASELINE B SUMMARY: Zeroed Out (existing CTC eliminated in budget)
    Scenario Year EITC Cost CTC Cost Total Cost Child Poverty Rate Children in Poverty
    Baseline 2025   $57.63M   $0.00M    $57.63M             28.71%          10,626,794
 EITC Reform 2025   $64.15M   $0.00M    $64.15M             28.71%          10,626,794
    Baseline 2026   $57.78M   $0.00M    $57.78M             28.29%          10,620,556
New CTC Only 2026   $57.78M  $85.78M   $143.56M             27.69%          10,396,840
  EITC + CTC 2026   $64.33M  $85.78M   $150.10M             27.69%          10,396,840

Exported to: dc_reforms_baseline_b_zeroed_out.csv


## Net Cost Comparison

In [41]:
print("\n" + "=" * 120)
print("NET COST ANALYSIS")
print("=" * 120)

print("\n" + "="*60)
print("BASELINE A: Relative to Current Law")
print("="*60)
print(f"\n2025 - EITC Enhancement:")
print(f"  EITC net cost: {format_currency(a1_eitc_budget['net_cost'])}")
print(f"  CTC net cost:  $0.00M (unchanged)")
print(f"  Total:         {format_currency(a1_eitc_budget['net_cost'])}")

print(f"\n2026 - New CTC Only:")
print(f"  EITC net cost: $0.00M (unchanged at 85%)")
print(f"  CTC net cost:  {format_currency(a2_ctc_budget['net_cost'])}")
print(f"  Total:         {format_currency(a2_ctc_budget['net_cost'])}")

print(f"\n2026 - Combined EITC + CTC:")
print(f"  EITC net cost: {format_currency(a3_eitc_budget['net_cost'])}")
print(f"  CTC net cost:  {format_currency(a3_ctc_budget['net_cost'])}")
print(f"  Total:         {format_currency(a3_total_cost)}")

print("\n" + "="*60)
print("BASELINE B: Relative to Zeroed Out CTC")
print("="*60)
print(f"\n2025 - EITC Enhancement:")
print(f"  EITC net cost: {format_currency(b1_eitc_budget['net_cost'])}")
print(f"  CTC net cost:  $0.00M (remains eliminated)")
print(f"  Total:         {format_currency(b1_eitc_budget['net_cost'])}")

print(f"\n2026 - New CTC Only:")
print(f"  EITC net cost: $0.00M (unchanged at 85%)")
print(f"  CTC net cost:  {format_currency(b2_ctc_budget['net_cost'])}")
print(f"  Total:         {format_currency(b2_ctc_budget['net_cost'])}")

print(f"\n2026 - Combined EITC + CTC:")
print(f"  EITC net cost: {format_currency(b3_eitc_budget['net_cost'])}")
print(f"  CTC net cost:  {format_currency(b3_ctc_budget['net_cost'])}")
print(f"  Total:         {format_currency(b3_total_cost)}")

print("\n" + "="*60)
print("Comparison to DC Council Estimates")
print("="*60)
print(f"\nFY26 (2025) EITC:")
print(f"  Council estimate: $17.42M")
print(f"  Our estimate:     {format_currency(a1_eitc_budget['net_cost'])}")

print(f"\nFY27 (2026) CTC (vs zeroed baseline):")
print(f"  Council estimate: $55.26M")
print(f"  Our estimate:     {format_currency(b3_ctc_budget['net_cost'])}")


NET COST ANALYSIS

BASELINE A: Relative to Current Law

2025 - EITC Enhancement:
  EITC net cost: $6.52M
  CTC net cost:  $0.00M (unchanged)
  Total:         $6.52M

2026 - New CTC Only:
  EITC net cost: $0.00M (unchanged at 85%)
  CTC net cost:  $68.96M
  Total:         $68.96M

2026 - Combined EITC + CTC:
  EITC net cost: $6.55M
  CTC net cost:  $68.96M
  Total:         $75.51M

BASELINE B: Relative to Zeroed Out CTC

2025 - EITC Enhancement:
  EITC net cost: $6.52M
  CTC net cost:  $0.00M (remains eliminated)
  Total:         $6.52M

2026 - New CTC Only:
  EITC net cost: $0.00M (unchanged at 85%)
  CTC net cost:  $85.78M
  Total:         $85.78M

2026 - Combined EITC + CTC:
  EITC net cost: $6.55M
  CTC net cost:  $85.78M
  Total:         $92.33M

Comparison to DC Council Estimates

FY26 (2025) EITC:
  Council estimate: $17.42M
  Our estimate:     $6.52M

FY27 (2026) CTC (vs zeroed baseline):
  Council estimate: $55.26M
  Our estimate:     $85.78M
